In [43]:
import pandas as pd
from dateutil import parser
from geopy import Nominatim
from geopy.distance import great_circle
import numpy as np
from numpy.linalg import norm
from time import sleep

import glob
import os

geolocator = Nominatim()

# used a geocoder to translate street addresses to coordinates
def locToCoords(location):
    sleep(1.1)
    coords = geolocator.geocode(location + " Charlottesville Virginia")
    if coords is not None and coords.longitude is not None:
        longitude = coords.longitude
        latitude = coords.latitude
        return(latitude, longitude)
    return("Did not process")

# Mall coordinates -  (38.031233, -78.482905) (38.029413, -78.477146)
def distToMall(posTuple):
    center = (38.030, -78.48)
    return(great_circle(center, posTuple).miles)

# pandas apply didn't want to work with a dataframe
# so this was my workaround don't judge pls
def dict_to_coords(value):
    try:
        return locdict[value]
    except:
        return np.nan
    
def fix_other_cvilles(dist):
    if(dist > 1000):
        return(.05)
    return(dist)

In [45]:
print(distToMall(locToCoords("1636 ST ANNES RD")))

1.8947960373522337


In [64]:
# loading data, normal stuff. Dropping duplicates based on location to reduce number of queries
ticket_path = os.path.join('..','data','parking_tickets','Parking_Tickets.csv')
data = pd.read_csv(ticket_path, low_memory=0)
data = data[data['DateIssued'].str.contains('2017')].sort_values(by=['DateIssued'])
#no_dup_data = data.drop_duplicates(['Location'])

In [ ]:
# querying the geocoder and saving the result
no_dup_data['Coords'] = no_dup_data["Location"].apply(locToCoords)
no_dup_data.to_csv('coords.csv')

In [59]:
data.head()

,RecordID,TicketNumber,DateIssued,StreetName,TimeIssued,StreetNumber,LicenseState,WaiverRequestDate,WaiverGrantedDate,AppealDate,AppealGrantedDate,ViolationDescription,AppealStatus,Location,LicensePlateAnon
363636,363637,87556602,2017-01-03T00:00:00.000Z,LEXINGTON AVE,12:57,400,VA,NaN,NaN,NaN,NaN,Overtime Parking,n/a,400 LEXINGTON AVE,65255
363333,363334,87512602,2017-01-03T00:00:00.000Z,W MAIN ST,13:06,500,VA,NaN,NaN,NaN,NaN,Overtime Parking,n/a,500 W MAIN ST,82086
12806,12807,00196009,2017-01-03T09:54:11.000Z,E MARKET ST,09:54,700,VA,NaN,NaN,NaN,NaN,Handicapped Space,n/a,700 E MARKET ST,109950
12807,12808,00196010,2017-01-03T09:57:42.000Z,E MARKET ST,09:57,700,VA,NaN,NaN,NaN,NaN,Handicapped Space,n/a,700 E MARKET ST,104027
12808,12809,00196011,2017-01-03T10:08:24.000Z,W MARKET ST,10:08,100,VA,NaN,NaN,NaN,NaN,Handicapped Space,n/a,100 W MARKET ST,109555


In [63]:
# turn these two columns into a dict so we can apply to the duplicates
ticket_path = os.path.join('..','data','parking_tickets','coords.csv')
no_dup_data = pd.read_csv(ticket_path, low_memory=0)
locdict = no_dup_data.set_index('Location')['Coords'].to_dict()

In [65]:
# filter out values that the geocoder couldn't parse
data = data[data['Location'].isin(locdict.keys())]

In [66]:
# apply the dict to the full list with duplicates
data['Coords'] = data['Location'].apply(dict_to_coords)
data = data[data['Coords'] != np.nan]
# distance squared, clipped at .025, which is right on the mall (getting rid of crazy low values)
data['DistMetric'] = data['Coords'].apply(distToMall)

In [ ]:
data.to_csv('ticket_with_loc.csv')

In [60]:
ticket_path = os.path.join('..','data','parking_tickets','ticket_with_loc.csv')
data = pd.read_csv(ticket_path, low_memory=0)
data['DistMetric'] = data['Coords'].apply(distToMall)

In [71]:
data = data[data["DistMetric"] < .5]

In [72]:
data.shape

(8021, 17)

In [74]:
data.to_csv('tickets_near_mall.csv')